In [1]:
# Step 1: Library Imports
from transformers import WhisperProcessor, WhisperForConditionalGeneration, AutoTokenizer
import torch
import soundfile as sf
import subprocess
import os
import time
from groq import Groq
from parler_tts import ParlerTTSForConditionalGeneration


Flash attention 2 is not installed


In [2]:
# Step 2: Audio Conversion Function
def convert_audio_to_wav(input_path, output_path="output.wav", target_sample_rate=16000):
    try:
        command = [
            "ffmpeg", "-i", input_path,
            "-ar", str(target_sample_rate),
            "-ac", "1",
            output_path
        ]
        subprocess.run(command, check=True)
        print(f"Converted {input_path} to {output_path} at {target_sample_rate} Hz.")
    except subprocess.CalledProcessError as e:
        print("An error occurred during conversion:", e)


In [3]:
# Step 3: Transcription Function
def transcribe_audio(audio_path, model_repo):
    processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
    model = WhisperForConditionalGeneration.from_pretrained(model_repo)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    
    audio_input, sampling_rate = sf.read(audio_path)
    
    if len(audio_input.shape) > 1:
        audio_input = audio_input.mean(axis=1)
    
    input_features = processor(audio_input, sampling_rate=sampling_rate, return_tensors="pt").input_features
    input_features = input_features.to(device)
    
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription


In [4]:
# Step 4: GROQ Chat Completion
def generate_chat_response(transcription):
    client = Groq(api_key='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')
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": f'''You are a medical assistant give a very friendly one line response for this query: {transcription} Remember to give a single reply in just one reply'''}],
        model="llama-3.2-1b-preview",
    )
    return chat_completion.choices[0].message.content


In [5]:
# Step 5: Text-to-Speech (TTS) Functionality
def synthesize_speech(prompt):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
    tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")
    
    description = "Laura Female Indian voice normal"
    input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio_arr = generation.cpu().numpy().squeeze()
    
    sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)


In [ ]:
# Step 6: Execution Cell
start_time = time.time()

# Specify your audio file path
input_audio_file = "sample.m4a"  
output_wav_file = "outputFile.wav"

# Convert audio to WAV
convert_audio_to_wav(input_audio_file, output_wav_file)

In [7]:

output_wav_file = "outputFile.wav"
# Transcribe the audio
model_repo = "shReYas0363/whisper-tiny-fine-tuned"
transcription = transcribe_audio(output_wav_file, model_repo)
print("Transcription:", transcription)

Transcription:  Hello, my check123


In [13]:


# Generate chat response
resps = generate_chat_response(transcription)
print("Chat Response:", resps.strip('"'))

# Synthesize speech

# Measure execution time
# end_time = time.time()
# print(f"The total time taken is: {end_time - start_time:.2f} seconds")

# # Cleanup
# os.remove(output_wav_file)


Chat Response: Get checked out, if not now, sooner!


In [14]:
import requests

url = "http://[::1]:5002/api/tts"
params = {
    "text": resps,
    "speaker_id": "p374",
    "style_wav": "",
    "language_id": ""
}

# Make the GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    with open('output_wav.wav', 'wb') as wav_file:
        wav_file.write(response.content)
    print("WAV file has been saved as 'output_wav.wav'.")
    
else:
    print(f"Error: {response.status_code} - {response.text}")

WAV file has been saved as 'output_wav.wav'.


e:\anaconda3\Lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).